In [3]:
import spinetools
import os
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from copy import deepcopy
import random
from plotly.subplots import make_subplots

np.set_printoptions(precision=2)


root_directory = r"D:\Documents\PolyMTL\3D_reconstruction\Lab_1\Datafiles_LAB1"
view_0 = 'PA0'
view_1 = 'LAT'

# Preamble
Please write the RMS calculation function here. You will need this function throughout the assignment, so craft it with care !

In [ ]:
def rms(reference:np.ndarray, prediction:np.ndarray) -> float:
    """
    Calculates RMS

    Parameters
    ----------
    reference : np.ndarray
        Reference value
    prediction : np.ndarray
        Calculated/predicted value

    Returns
    -------
    float
        RMS metric
    """
    if reference.shape != prediction.shape:
        raise ValueError('Please provide arrays that are shaped the same ! Reference has shape {refshape} whereas prediction has shape {predshape}'.format(refshape = reference.shape, predshape = prediction.shape))
    rms_value = 0.
    if len(reference.shape) == 2: # Array of shape N, 3:
        rms_value= np.sqrt(np.power(np.linalg.norm(reference-prediction, axis=1),2)).mean()
    else: # Array of shape N, :
        rms_value = np.sqrt(np.power(reference-prediction,2)).mean()
    return rms_value

# Question 1.1
Write the code for finding the over determined system matrix A. Then use the dlt function from spinetools.solver to extract the L parameters vector for each view.

In [4]:
beads_data = spinetools.io.process_files(root_directory)
# ? Filter out non redundant beads
unique_beads = beads_data['2d']['bead'].tolist()
unique_beads = [x.replace('C', 'A') for x in unique_beads]
unique_beads = [x.replace('D', 'B') for x in unique_beads]

# ? Count values
value_count = {}
for k in unique_beads:
    if k in value_count.keys():
        value_count[k] += 1
    else:
        value_count[k] = 1

# ? Exclude the ones that aren't visible from everywhere
for k, v in zip(list(value_count.keys()), list(value_count.values())):
    if v == 1:
        print('Dropped', k)
        del value_count[k]

unique_beads = list(value_count.keys())
unique_beads.extend([x.replace('A', 'C') if 'A' in x else x.replace('B', 'D') for x in unique_beads])
beads_data['2d'] = beads_data['2d'][beads_data['2d']['bead'].isin(unique_beads)]
beads_data['3d'] = beads_data['3d'][beads_data['3d']['bead'].isin(unique_beads)]

Dropped A_4_1
Dropped B_3_5


In [ ]:
from spinetools.data import select_view, join_dataframes

def get_a(view:str, data:dict) -> np.ndarray:
    """
    Uses the view name and existing data for pre calculating the system matrix for calculating the 11 DLT coefficients

    Parameters
    ----------
    view : str
        View name
    data : dict
        Data as a dict, with two keys ['2d', '3d']. Each key is associated to a table giving points location in 2d/3d, bead identifier and view name.

    Returns
    -------
    np.ndarray
        A matrix, for the DLT
    """
    A = []
    joint_data = join_dataframes(select_view(data['2d'], view), data['3d'], 'bead')
    # TODO : Using the equations from Appendix 1, write a function to turn pairs of 2D/3D beads data to an overdeterminated system.
    # HINT 1 : we want to end up with some system AM = 0 where M is a vector containing [L0, L1... L10, 1]. So you just need to return A !
    # HINT 2 : I gave you 'joint_data'. It's a dataframe, so check the columns names !

    

    return np.array(A)

In [ ]:
l_view0 = spinetools.solver.dlt(get_a('Beads2D_'+view_0, beads_data))
l_view1 = spinetools.solver.dlt(get_a('Beads2D_'+view_1, beads_data))

In [ ]:
print("DLT parameters for %s : "%view_0, l_view0)
print("DLT parameters for %s : "%view_1, l_view1)

# Question 1.2

In [ ]:
def get_parameters(dlt_params:np.ndarray) -> dict:
    """
    Outputs a dictionnary containing the intrinsic and extrinsic parameters of the system based on the DLT parameters.

    Parameters
    ----------
    dlt_params : np.ndarray
        _description_

    Returns
    -------
    dict
        Dict with keys ['u0', 'v0', 'c_u', 'c_v', 'R', 'source_coordinates']
    """
    out = {}
    # TODO : Using the equations from Appendix 2, write a function to calculate intrinsic and extrinsic parameters of a system based on the DLT parameters
    return out

print('View %s :'%view_0)
for k, v in get_parameters(l_view0).items():
    print('\t {variable} = {value}'.format(variable = k, value = v))
print('-------------------------------------------------------------')
print('View %s :'%view_1)
for k, v in get_parameters(l_view1).items():
    print('\t {variable} = {value}'.format(variable = k, value = v))

# Question 1.3

In [ ]:
def calculate_3d_point(u0:float, v0:float, u1:float, v1:float, L0:np.ndarray, L1:np.ndarray) -> np.ndarray:
    """
    Calculates a 3D point from 2 2D coordinates pairs and L0 & L1 DLT calibration coefficients

    Parameters
    ----------
    u0 : float
        point 1 x-coordinate
    v0 : float
        point 1 y-coordinate
    u1 : float
        point 2 x-coordinate
    v1 : float
        point 2 y-coordinate
    L0 : np.ndarray
        DLT coefficients for view 1
    L1 : np.ndarray
        DLT coefficients for view 2
    """
    A = []
    # TODO : using Appendix A, write a function that takes 2 pairs of 2D coordinates and 2 11-parameters vectors as an input to return the corresponding 3D point. 
    # HINT 1 : It's the exact same problem as in question 1.1, but now the unknowns are x,y and z !
    # HINT 2 : Refactor the pair of equation in order to have a 4 equations system (2 equations per view, 2 views) for finding a triplet of coordinates : AM = 0 where M = [x, y, z, 1]
    return spinetools.solver.dlt(np.array(A))

In [ ]:
spine = spinetools.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat'))
colors = px.colors.qualitative.Alphabet
fig = spinetools.render.Plot3D()

for i, v_name in enumerate(spine.vertebrae):
    # TODO : for each vertebra, calculate the location of each anatomical landmark and add its x,y,z coordinates to vert_x, vert_y, vert_z. Each vertebra will then be added to the 3D plot and plotted in the end.
    vert = spine.get(v_name).data # Returns the vertebra data
    """
    HINT : the data is a table containing the landmark name and the x,y coordinates for each view (so 5 columns)
    HINT : Print the variable vert, and add a 'break' instruction to get out of the loop
    For each vertebra, you should have 6 points. Each point is linked to 2 pairs of 2D coordinates, one in each view. These coordinates, with the two 11-DLT parameters will allow you to calculate the 3D position of the 6 anatomical landmarks on the vertebra.
    """
    vert_x, vert_y, vert_z = [], [], []
    """
    PSEUDOCODE
    for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append x_point to vert_x
        Append y_point to vert_y
        Append z_point to vert_z
    """
    fig.scatter_vertebrae(vert_x, vert_y, vert_z, v_name, colors[i]) # Adds the landmarks to the figure
SPINE_3D_POINTS = fig.points # Access all the figure points as an array this way, will be useful in the rest of this assignment
fig.show() # Shows the figure

# Question 2.1

Here I recommend you write a function that samples `n_beads` in the full set, and returns the subset. A code skeleton and some hints can be found below.

In [ ]:
def sample_n_beads(original_beads:dict, n_beads:int) -> dict:
    """
    Samples n_beads beads from the original beads set

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the whole set of calibration beads
    n_beads : int
        Number of beads to select

    Returns
    -------
    dict
        Dictionnary containing information on 2D and 3D position of the subsampled of calibration beads
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    selected_beads = []
    """
    Let's modify our output until we reach the desired number of beads.
    You want n beads sampled across both plates A & B (remember that plates C & D are just A & B rotated):
    1. So you're gonna sample n_beads // 2 beads on plate A, and n_beads - (n_beads // 2) beads on plate B.
    2. Add all these beads' names in the selected_beads list.
    3. Then for each A & B bead, you want to add it's counterpart of plate C & D.
    The selection step is already done :)
    """
    output['2d'] = output['2d'][output['2d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    output['3d'] = output['3d'][output['3d']['bead'].isin(selected_beads)].sort_values(by = 'bead', ignore_index = True)
    return output

In [ ]:
# Let's increase the number of calibration beads gradually
spine = spinetools.structures.Spine(os.path.join(root_directory,'Vertebrae2D.mat')) # Initialize the vertebrae structure

# Create a dictionnary to store the results. Access a dictionnary key with brackets, for instance rms_measurements['x']
rms_measurements = {
        'x' : [],
        'y' : [],
        'z' : [],
        '3d' : [],
        'n_beads':[]
    }

"""
PSEUDOCODE
computed_points_3d <- list
for n_beads in [1...N], do:
    beads_subset <- sample_n_beads(beads_data, n_beads)
    A_view_0 <- get_a(view_0, beads_subset)
    l_view_0 <- dlt(A_view_0)
    A_view_1 <- get_a(view_1, beads_subset)
    l_view_1 <- dlt(A_view_1)
    for each vertebra in vertebrae:
        for each anatomical_landmark in vertebra_points:
        x_point, y_point, z_point <- calculate_3d_point(anatomical_landmark, l_view0, l_view1)
        Append (x_point, y_point, z_point) to computed_points_3d
    for each axis in ['x', ..., '3d']:
        Append rms(SPINE_3D_POINTS, computed_points_3d) to rms_measurements[axis]
        Append n_beads to rms_measurements['n_beads']
"""

# Plotly interlude

In [ ]:
"""
Here is a basic plotly example. Use it to plot your RMS values !
"""
fig = go.Figure()
fig.update_layout(title = 'Various functions plot',
                  yaxis_title = 'f(x)',
                  xaxis_title = 'x')
x_values = np.linspace(0, 10, 1001)
y_values_0 = np.sin(x_values) * np.sin(2*x_values)
y_values_1 = np.cos(x_values) * np.sin(0.5*np.power(x_values, 2))
fig.add_trace(go.Scatter(
    x = x_values,
    y = y_values_0,
    name = 'sin(x) * sin(2x)'
))
fig.add_trace(go.Scatter(
    x = x_values,
    y = y_values_1,
    name = 'cos(x) * sin(0.5 * x^2))'
))
fig.show()

In [ ]:
# TODO : On the same graph, plot all your RMS values wrt. the number of used calibration beads !

# Question 2.2

Here are 4 sets of 8 calibration beads. For each set, determine which one is which (same plate, small volume, medium volume, large volume) and plot calibration error curves.  
Don't hesitate to use the function `plot_selected_beads` from `spinetools.render`to vizualize the selected beads to sort out the subsets !

In [ ]:
set_0 = ['A_1_6', 'A_4_6', 'A_1_2', 'A_4_2', 'B_1_5', 'B_5_5', 'B_1_1', 'B_5_1']
set_1 = ['B_1_1', 'B_2_2', 'B_4_2', 'B_5_3', 'B_1_4', 'B_2_5', 'B_4_3', 'B_5_5']
set_2 = ['A_1_4', 'A_2_4', 'A_1_5', 'A_2_5', 'B_1_3', 'B_2_3', 'B_1_4', 'B_2_4']
set_3 = ['A_1_4', 'A_3_4', 'A_1_5', 'A_3_5', 'B_1_3', 'B_3_3', 'B_1_4', 'B_3_4']

TODO : same thing as before, but now you have to select the provided beads ! Look at the function `sample_n_beads` from Q2.1 if you need some help on how to do this.

# Question 2.3

Using the **small** calibration volume from previous question, plot reconstruction error with respect to the center of gravity of the calibration volume !

# Question 3.1 

Here you have to code the function that allow us to add noise to the calibration beads coordinates. Hint : you should take a look at the documentation of `np.random.normal` :)

In [ ]:
def add_noise_2d(original_beads:dict, std:float) -> dict:
    """
    Adds noise to the 2D coordinates of the beads

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the a set of calibration beads
    std : float
        Standard deviation for the noise added.

    Returns
    -------
    dict
        Noised beads set
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    # TODO : select 2D beads and add noise to them
    
    return output

def add_noise_3d(original_beads:dict, std:float) -> dict:
    """
    Adds noise to the 3D coordinates of the beads

    Parameters
    ----------
    original_beads : dict
        Dictionnary containing information on 2D and 3D position of the a set of calibration beads
    std : float
        Standard deviation for the noise added.

    Returns
    -------
    dict
        Noised beads set
    """
    output = deepcopy(original_beads) # ? First we deepcopy the original beads data, to avoid modifying it by mistake
    # TODO : select 3D beads and add noise to them
    
    return output

# Question 3.2

# Question 4.1

# Question 4.2

This question is a bit tricky, as you have to get measurements for couples of values of standard deviation for 2D and 3D noise. A line plot is not very adapted for this, but we could use a heatmap ! Here is a small example :

In [ ]:
fig = go.Figure()
heatmap = np.random.rand(11, 11)
fig.add_trace(
    go.Heatmap(
        z = heatmap,
        x = np.linspace(0., 10., 11),
        y = np.linspace(0., 10., 11),
        text = heatmap.round(2),
        texttemplate="%{text}",
        colorscale='turbo'
    ))
fig.update_xaxes(title_text='Axis 1')
fig.update_yaxes(title_text='Axis 2')
fig.update_layout(title = 'Plot title')
fig.update_traces(showscale=False)
fig.show()